In [ ]:
import google.generativeai as genai
import speech_recognition as sr
import pyttsx3
import os
from datetime import date
import time
import streamlit as st
from openai import OpenAI

client = OpenAI()

# Set up Streamlit UI elements
st.title("Voice Chatbot")
st.write("Talk to the AI assistant")

# Set up the Google Gemini API key
genai.configure(api_key="Your-API-key")

today = str(date.today())

# Initialize the TTS engine (for text-to-speech)
engine = pyttsx3.init()
engine.setProperty("rate", 190)  # speaking rate
voices = engine.getProperty("voices")
engine.setProperty("voice", voices[1].id)  # 0 for male; 1 for female

# Model of Google Gemini API
model = genai.GenerativeModel("gemini-pro")

# Select to use OpenAI's text to speech API
openaitts = True


def speak_text(text):
    global openaitts

    if openaitts:
        response = client.audio.speech.create(
            model="tts-1",
            voice="nova",  # alloy: man; nova: woman
            input=text,
        )

        fname = "output.mp3"
        response.write_to_file(fname)

        # Streamlit audio playback
        with open(fname, "rb") as audio_file:
            audio_bytes = audio_file.read()
            st.audio(audio_bytes, format="audio/mp3")

    else:
        engine.say(text)
        engine.runAndWait()


# Save conversation to a log file
def append2log(text):
    global today
    fname = "chatlog-" + today + ".txt"
    with open(fname, "a") as f:
        f.write(text + "\n")


# Main function for conversation
def main():
    global talk, today, model

    rec = sr.Recognizer()
    mic = sr.Microphone()
    rec.dynamic_energy_threshold = False
    rec.energy_threshold = 400

    sleeping = True

    while True:

        with mic as source1:
            rec.adjust_for_ambient_noise(source1, duration=0.5)

            st.write("Listening...")

            try:
                audio = rec.listen(source1, timeout=10, phrase_time_limit=15)

                text = rec.recognize_google(audio)

                # AI is in sleeping mode
                if sleeping:
                    # User must start the conversation with the wake word "Jack"
                    if "jack" in text.lower():
                        request = text.lower().split("jack")[1]

                        sleeping = False
                        append2log(f"_" * 40)
                        talk = []
                        today = str(date.today())

                        if len(request) < 5:
                            speak_text("Hi, there, how can I help?")
                            append2log(f"AI: Hi, there, how can I help? \n")
                            continue

                    else:
                        continue

                # AI is awake
                else:
                    request = text.lower()

                    if "that's all" in request:
                        append2log(f"You: {request}\n")
                        speak_text("Bye now")
                        append2log(f"AI: Bye now. \n")
                        st.write("Bye now")
                        sleeping = True
                        continue

                    if "jack" in request:
                        request = request.split("jack")[1]

                    append2log(f"You: {request}\n ")

                    st.write(f"You: {request}\n AI: ")

                    talk.append({"role": "user", "parts": [request]})

                    response = model.generate_content(talk, stream=True)

                    for chunk in response:
                        st.write(chunk.text, end="")
                        speak_text(chunk.text.replace("*", ""))

                    talk.append({"role": "model", "parts": [response.text]})

                    append2log(f"AI: {response.text } \n")

            except Exception as e:
                continue


if __name__ == "__main__":
    main()